In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from lifetimes.utils import summary_data_from_transaction_data
import numpy as np
import datetime
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from lifetimes import ModifiedBetaGeoFitter
from lifetimes import GammaGammaFitter
import dask.dataframe as dd
from dask.multiprocessing import get
from collections import Counter
from sklearn.model_selection import train_test_split
import gc
import joblib
import boto3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras
config = tf.ConfigProto(device_count={"CPU": 20})
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))
from keras.models import Sequential
from keras.layers import Dense
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Dropout
from sklearn.utils import class_weight
from sklearn.model_selection import RandomizedSearchCV, KFold
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras import layers
from keras import Input
from keras.layers import BatchNormalization
from sklearn.model_selection import ParameterSampler
from scipy.stats.distributions import expon
from keras.models import load_model
import io
import json
from keras_preprocessing.text import tokenizer_from_json

debug = True

Using TensorFlow backend.


In [2]:
def _write_dataframe_to_csv_on_s3(df, path_s3):
    import boto3
    from io import StringIO
    """ Write a dataframe to a CSV on S3 """
    a = path_s3.split('//')
    b = a[1].split('/')
    bucket = b[0]
    c = path_s3.split(bucket+'/')
    path = c[1]
    
    buffer = StringIO()
    df.to_csv(buffer,index=False)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket, path).put(Body=buffer.getvalue())
    return None

In [2]:
def asigno_quintiles_6(x):
    step = int(x.shape[0]*0.1) # segmentos de 10%
    
    l_ = []
    for i in range(6):  # asigno los primeros 50% en segmentos de 10%
        l_ = l_ + ["q"+str(i+1)]*step    
    
    step2 = x.shape[0] - len(l_)
    l_ = l_ + ["q6"]*step2
    return l_

In [13]:
if debug == True:
    cores = 2
    n_iter=2
else:
    cores = cpu_count()
    n_iter = 30

In [14]:
if(debug == False):
    summary_cal = pd.read_csv('s3://fda-labs/ltv-ml/ML/preproc_data_train.csv.gzip')  # Because in the ETL we pickled the pandas dataframe
else:
    summary_cal = pd.read_csv('s3://fda-labs/ltv-ml/ML/sample_Preproc_MLA.csv')  # Because in the ETL we pickled the pandas dataframe

In [15]:
summary_cal = pd.merge(summary_cal,pd.read_csv("s3://fda-labs/ltv-ml/ML/summary_visitas_train.csv",sep = ","),on="cust",how="left")

In [16]:
tokenizer = Tokenizer(num_words=50000) # le pongo mucha cantidad para que tome todas las catergorias ;)
summary_cal["phrase"] = summary_cal["phrase"].fillna('')
tokenizer.fit_on_texts(summary_cal["phrase"])

X_seq_train = tokenizer.texts_to_sequences(summary_cal["phrase"])
vocab_size = len(tokenizer.word_index) + 1

maxlen = 450
X_seq_train = pad_sequences(X_seq_train, padding='pre', maxlen=maxlen)  

In [8]:
train_cols_struc = [ 'frequency', 'recency', 'T', 'monetary_value', 'my_recency', 'n_purchases_pred', 'p_alive',
                       'montary_sales_forecast', 'clv_pareto', 'target_pareto','IPT_CV', 'money_sum',
                       'money_CV', 'money_mean', 'money_std', 'money_max', 'money_min',
                       'L1_1000', 'L1_1039', 'L1_1051', 'L1_1071', 'L1_1132', 'L1_1144',
                       'L1_1168', 'L1_1182', 'L1_1246', 'L1_1276', 'L1_1367', 'L1_1368',
                       'L1_1384', 'L1_1403', 'L1_1430', 'L1_1499', 'L1_1574', 'L1_1648',
                       'L1_1798', 'L1_1953', 'L1_2547', 'L1_3025', 'L1_3937', 'L1_407134',
                       'L1_409431', 'L1_5725', 'L1_5726', 'female', 'male', 'Account Money',
                       'Bank Transfer', 'Credit Card', 'Debit Card', 'Ticket',
                       'digital_currency', '01.Menor de 18 años', '02.Entre 18 y 25 años',
                       '03.Entre 26 y 30 años', '04.Entre 31 y 35 años',
                       '05.Entre 36 y 40 años', '06.Entre 41 y 55 años',
                       '07.Entre 56 y 65 años', '08.Mayor de 65 años']

In [18]:
# encode class values as integers
Y_train = summary_cal["quintil_true"].values
encoder = LabelEncoder()
encoder.fit(Y_train)
encoded_Y = encoder.transform(Y_train)
y_train = np_utils.to_categorical(encoded_Y)

In [19]:
stdc = StandardScaler()
stdc_trained = stdc.fit(summary_cal[train_cols_struc])
X_train_struc = stdc_trained.transform(summary_cal[train_cols_struc])

In [20]:
# mezclo loss datos para que no overfite!
shuffle_index = np.random.choice(X_seq_train.shape[0], X_seq_train.shape[0], replace=False)
X_seq_train = X_seq_train[shuffle_index,:]
X_train_struc = X_train_struc[shuffle_index,:]
y_train = y_train[shuffle_index,:]
Y_train = Y_train[shuffle_index]

In [3]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0.01, 
                     patience=10, 
                   verbose=1, 
                   restore_best_weights=True)
    
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(Y_train), Y_train)

weights = dict(zip(list(range(len(weights))), weights))

def train_model(emb=32,conv=32,kernel = 3,nn=10, lr=0.01, decay=0., l1=0.01, l2=0.01,
                 dropout=0,model_out = False):
    print(emb,conv,kernel,nn,lr,decay,l1,l2,dropout)
    '''This is a model generating function so that we can search over neural net 
    parameters and architecture'''

    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999,  decay=decay)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
    
    # Rama visitas
    input_visits = Input(shape=(X_seq_train.shape[1],))
    # Define encoding layer
    embedding = layers.Embedding(vocab_size, int(emb), input_length=maxlen)(input_visits)
    next_seq = layers.Conv1D(int(conv), int(kernel), activation='relu')(embedding)
    next_seq = layers.GlobalMaxPooling1D()(next_seq)
    if(nn > 0):
        next_seq = layers.Dense(int(nn), activation='relu', kernel_regularizer=reg)(next_seq)
        if(dropout > 0):
            next_seq =  layers.Dropout(dropout)(next_seq)
        
        next_seq = BatchNormalization()(next_seq)
        
    # Rama daatos estructurados    
    input_structured = Input(shape=(X_train_struc.shape[1],))
    next_dense =  layers.Dense(50, activation='relu', kernel_regularizer=reg)(input_structured)
    if(dropout > 0):
        next_dense =  layers.Dropout(dropout)(next_dense)
    next_dense = BatchNormalization()(next_dense)
    
    next_dense =  layers.Dense(50, activation='relu', kernel_regularizer=reg)(next_dense)
    if(dropout > 0):
        next_dense =  layers.Dropout(dropout)(next_dense)
    next_dense = BatchNormalization()(next_dense)
    
    concat = layers.concatenate([next_seq,next_dense], axis=-1)
    out = layers.Dense(6, activation='softmax')(concat)

    model = Model(inputs = [input_visits,input_structured], outputs = out)
        
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[f1])
    
    hist = model.fit([X_seq_train,X_train_struc],y_train,class_weight = weights, validation_split = 0.2,
                 callbacks=[es],verbose=10,epochs = 50, batch_size=512)

    if(model_out == True):
        return model
    
    return hist

def second_model(model_out):
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999,  decay=1e-9)
    reg = keras.regularizers.l1_l2(l1=0.0001, l2=0.0001)
    
    # Rama visitas
    input_visits = Input(shape=(X_seq_train.shape[1],))
    # Define encoding layer
    embedding = layers.Embedding(vocab_size, int(32), input_length=maxlen)(input_visits)
    next_seq = layers.Conv1D(int(16), int(3), activation='relu')(embedding)
    next_seq = layers.GlobalMaxPooling1D()(next_seq)

    # Rama structurado
    input_structured = Input(shape=(X_train_struc.shape[1],))
    next_dense =  layers.Dense(100, activation='relu', kernel_regularizer=reg)(input_structured)
    next_dense =  layers.Dropout(0.5)(next_dense)
    next_dense = BatchNormalization()(next_dense)

    next_dense =  layers.Dense(100, activation='relu', kernel_regularizer=reg)(input_structured)
    next_dense =  layers.Dropout(0.5)(next_dense)
    next_dense = BatchNormalization()(next_dense)
    
    next_dense =  layers.Dense(30, activation='relu', kernel_regularizer=reg)(input_structured)
    next_dense =  layers.Dropout(0.5)(next_dense)
    next_dense = BatchNormalization()(next_dense)
    
    concat = layers.concatenate([next_seq,next_dense], axis=-1)
    out = layers.Dense(6, activation='softmax')(concat)

    model = Model(inputs = [input_visits,input_structured], outputs = out)
        
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[f1])
    
    hist = model.fit([X_seq_train,X_train_struc],y_train,class_weight = weights, validation_split = 0.2,
                 callbacks=[es],verbose=10,epochs = 50, batch_size=512)

    if(model_out == True):
        return model

def third_model(model_out):
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999,  decay=1e-9)
    reg = keras.regularizers.l1_l2(l1=0.0001, l2=0.0001)
    
    # Rama visitas
    input_visits = Input(shape=(X_seq_train.shape[1],))
    # Define encoding layer
    embedding = layers.Embedding(vocab_size, int(32), input_length=maxlen)(input_visits)
    next_seq = layers.Conv1D(int(16), int(3), activation='relu')(embedding)
    next_seq = layers.GlobalMaxPooling1D()(next_seq)
    next_seq = layers.Dense(int(100), activation='relu', kernel_regularizer=reg)(next_seq)
    next_seq =  layers.Dropout(0.5)(next_seq)
    next_seq = BatchNormalization()(next_seq)
    next_seq = layers.Dense(int(30), activation='relu', kernel_regularizer=reg)(next_seq)
    next_seq =  layers.Dropout(0.5)(next_seq)
    next_seq = BatchNormalization()(next_seq)    
    
    # Rama structurado
    input_structured = Input(shape=(X_train_struc.shape[1],))
    next_dense =  layers.Dense(100, activation='relu', kernel_regularizer=reg)(input_structured)
    next_dense =  layers.Dropout(0.5)(next_dense)
    next_dense = BatchNormalization()(next_dense)

    next_dense =  layers.Dense(100, activation='relu', kernel_regularizer=reg)(input_structured)
    next_dense =  layers.Dropout(0.5)(next_dense)
    next_dense = BatchNormalization()(next_dense)
    
    next_dense =  layers.Dense(30, activation='relu', kernel_regularizer=reg)(input_structured)
    next_dense =  layers.Dropout(0.5)(next_dense)
    next_dense = BatchNormalization()(next_dense)
    
    concat = layers.concatenate([next_seq,next_dense], axis=-1)
    out = layers.Dense(6, activation='softmax')(concat)

    model = Model(inputs = [input_visits,input_structured], outputs = out)
        
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[f1])
    
    hist = model.fit([X_seq_train,X_train_struc],y_train,class_weight = weights, validation_split = 0.2,
                 callbacks=[es],verbose=10,epochs = 50, batch_size=512)

    if(model_out == True):
        return model

NameError: name 'Y_train' is not defined

In [ ]:
best_param = {'conv': 16.0,
 'decay': 1e-09,
 'dropout': 0.0,
 'emb': 32.0,
 'kernel': 3.0,
 'l1': 0.01,
 'l2': 0.001,
 'lr': 0.01,
 'nn': 100.0,
 'model_out':True             
}

model = train_model(**best_param)
model.save("model.h5")

In [ ]:
joblib.dump(stdc_trained,"std.sav")

tokenizer_json = tokenizer.to_json()
with io.open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

s3 = boto3.client('s3')
s3.upload_file("model.h5", "fda-labs", "ltv-ml/Embedding/model1.h5")
s3.upload_file("std.sav", "fda-labs", "ltv-ml/Embedding/std.sav")
s3.upload_file("tokenizer.json", "fda-labs", "ltv-ml/Embedding/tokenizer.json")

In [ ]:
model = second_model(True)
model.save("model.h5")
s3.upload_file("model.h5", "fda-labs", "ltv-ml/Embedding/model2.h5")

In [ ]:
model = third_model(True)
model.save("model.h5")
s3.upload_file("model.h5", "fda-labs", "ltv-ml/Embedding/model3.h5")

In [22]:
# learning algorithm parameters
lr=[1e-2, 1e-3]
decay=[1e-6,1e-9,0]

# numbers of layers
nn = [0,10,100]
emb = [10,32,50]
conv = [8,16,32,64]
kernel =[3,5] 


# dropout and regularisation
dropout = [0,0.5]
l1 = [0, 0.01, 0.001]
l2 = [0, 0.01, 0.001]

# dictionary summary
param_grid = dict(
                    emb = emb, conv = conv, kernel = kernel, nn=nn, lr=lr, decay=decay, l1=l1, 
                    l2=l2, dropout=dropout                  
                 )

print("Cantidad de parametros:",sum([len(x) for x in param_grid.values() if(type(x) != int)]))

Cantidad de parametros: 25


In [72]:
rng = np.random.RandomState(0)
param_list = list(ParameterSampler(param_grid, n_iter=n_iter,
                                   random_state=rng))

l_hist = []
for params in param_list:
    params.update({"score":max(train_model(**params).history["val_f1"])})
    l_hist.append(params)

10 32 5 0 0.01 1e-06 0.01 0 0
Train on 204385 samples, validate on 51097 samples
Epoch 1/2
Epoch 2/2
32 16 3 10 0.01 1e-06 0.001 0 0.5
Train on 204385 samples, validate on 51097 samples
Epoch 1/2
Epoch 2/2


In [76]:
resu = pd.DataFrame(l_hist)
_write_dataframe_to_csv_on_s3(resu,"s3://fda-labs/ltv-ml/Embedding/modelPerformance.csv")

best_param = resu.loc[resu['score'].idxmax()].to_dict()
print(best_param)
best_param.pop('score', None)

model = train_model(**best_param,model_out = True)
model.save("model.h5")

joblib.dump(stdc_trained,"std.sav")

tokenizer_json = tokenizer.to_json()
with io.open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

s3 = boto3.client('s3')
s3.upload_file("model.h5", "fda-labs", "ltv-ml/Embedding/model.h5")
s3.upload_file("std.sav", "fda-labs", "ltv-ml/Embedding/std.sav")
s3.upload_file("tokenizer.json", "fda-labs", "ltv-ml/Embedding/tokenizer.json")

# dependencies = {
#     'f1': f1
# }

# load_model("model.h5",custom_objects=dependencies).summary()


# with open('tokenizer.json') as f:
#     data = json.load(f)
#     tokenizer = tokenizer_from_json(data)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_45 (InputLayer)           (None, 60)           0                                            
__________________________________________________________________________________________________
input_44 (InputLayer)           (None, 450)          0                                            
__________________________________________________________________________________________________
dense_75 (Dense)                (None, 50)           3050        input_45[0][0]                   
__________________________________________________________________________________________________
embedding_23 (Embedding)        (None, 450, 10)      106850      input_44[0][0]                   
__________________________________________________________________________________________________
batch_norm

## Test

In [4]:
summary_cal = pd.read_csv('s3://fda-labs/ltv-ml/ML/preproc_data_test.csv.gzip') 

In [5]:
summary_cal = pd.merge(summary_cal,pd.read_csv("s3://fda-labs/ltv-ml/ML/summary_visitas_test.csv",sep = ","),on="cust",how="left")

In [6]:
s3 = boto3.client('s3')
s3.download_file("fda-labs", "ltv-ml/Embedding/std.sav","std.sav")
stdc_trained = joblib.load("std.sav")

s3.download_file("fda-labs", "ltv-ml/Embedding/model1.h5","model1.h5")
s3.download_file("fda-labs", "ltv-ml/Embedding/tokenizer.json","tokenizer.json")

dependencies = {
    'f1': f1
}

model = load_model("model1.h5",custom_objects=dependencies)

encoder = LabelEncoder()
encoder.fit(summary_cal.quintil_true)

with open('tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [10]:
summary_cal["phrase"] = summary_cal["phrase"].fillna('')
#tokenizer.fit_on_texts(summary_cal["phrase"])
X_seq_train = tokenizer.texts_to_sequences(summary_cal["phrase"])
vocab_size = len(tokenizer.word_index) + 1
print("paso")
maxlen = 450
X_seq_train = pad_sequences(X_seq_train, padding='pre', maxlen=maxlen)      
print("paso")    
X_train_struc = stdc_trained.transform(summary_cal[train_cols_struc]) 
y = np_utils.to_categorical(encoder.transform(summary_cal.quintil_true))

In [11]:
preds = model.predict([X_seq_train,X_train_struc])

In [ ]:
summary_cal[["q1","q2","q3","q4","q5","q6"]] = pd.DataFrame(preds)
apply_me = summary_cal

def asigno_quintiles_1(x,step,q):    
    step2 = x.shape[0] - (step)
    l_ = ["q"+q]*step + ["non_q"]*step2
    return l_ 

apply_me["pred_q"] = "non_q"
resu = pd.DataFrame([])
step1 = int(apply_me.shape[0]*0.1)
step2 = int(apply_me.shape[0]*0.5)

for i in range(4):
    df = apply_me.loc[apply_me.pred_q == "non_q"]
    df = df.sort_values(by = "q"+str(i+1),ascending = False)
    df["pred_q"] = asigno_quintiles_1(df,step1,str(i+1))
    df = df.loc[df.pred_q != "non_q"]
    apply_me.loc[apply_me.cust.isin(df.cust.values),"pred_q"] = "q"+str(i+1)

df = apply_me.loc[apply_me.pred_q == "non_q"]
df = df.sort_values(by = "q6",ascending = False)
df["pred_q"] = asigno_quintiles_1(df,step2,"6")
df = df.loc[df.pred_q != "non_q"]
apply_me.loc[apply_me.cust.isin(df.cust.values),"pred_q"] = "q6"

apply_me.loc[apply_me.pred_q == "non_q","pred_q"] = "q5"

In [ ]:
confusion = confusion_matrix(apply_me["quintil_true"],apply_me["pred_q"])
for i in range(confusion.shape[0]):
    confusion[i,:] = 100*confusion[i,:]/confusion.sum(axis = 1)[i]
    
labels = ['q1', 'q2','q3','q4','q5','q6']
plt.figure(figsize=(10,10))
ax= plt.subplot()
sns.heatmap(confusion,annot = True,center = 1,ax=ax,cmap='bwr')
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix Modelo Dense'); 
ax.xaxis.set_ticklabels(labels); ax.yaxis.set_ticklabels(labels);

In [74]:
# import keras
# import pydotplus
# from keras.utils.vis_utils import model_to_dot
# import pydot

# keras.utils.vis_utils.pydot = pydot

# plot_model(net, show_shapes=True, to_file='model.png')
# # apt-get install graphviz

In [6]:
!aws s3 ls s3://fda-labs/ltv-ml/Embedding/

2019-07-04 23:35:05   13781183 doc2vec_10_12
2019-07-04 20:08:17   15115133 doc2vec_20_8
2019-07-05 03:06:43   16721256 doc2vec_32_12
2019-07-05 06:36:22   16734868 doc2vec_32_8
2019-07-03 15:15:46      66958 model.png
2019-07-05 14:27:12    4496000 model1.h5
2019-07-05 14:38:53    4396856 model2.h5
2019-07-05 14:55:57    4482408 model3.h5
2019-07-04 16:31:31        711 modelPerformance.csv
2019-07-05 14:27:12       2014 std.sav
2019-07-05 14:27:12    1260216 tokenizer.json


In [ ]:
labels = ['q1', 'q2','q3','q4','q5','q6']
plt.figure(figsize=(10,10))
ax= plt.subplot()
sns.heatmap(confusion,annot = True,center = 1,ax=ax,cmap='bwr')
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix Modelo Dense'); 
ax.xaxis.set_ticklabels(labels); ax.yaxis.set_ticklabels(labels);

plt.savefig("confusion_matrix.png")


In [8]:
!aws s3 cp s3://fda-labs/ltv-ml/Dense/ModelPerformance.csv read.csv

download: s3://fda-labs/ltv-ml/Dense/ModelPerformance.csv to ./read.csv


In [11]:
summary_cal.head()

,cust,frequency,recency,T,monetary_value,date_min,date_max,my_recency,n_purchases_pred,p_alive,...,Ticket,digital_currency,01.Menor de 18 años,02.Entre 18 y 25 años,03.Entre 26 y 30 años,04.Entre 31 y 35 años,05.Entre 36 y 40 años,06.Entre 41 y 55 años,07.Entre 56 y 65 años,08.Mayor de 65 años
0,137310278,77.0,78.0,78.0,2608.509221,2016-12-20,2018-06-20,0,288.677877,0.998795,...,0,0,0,0,0,0,0,1,0,0
1,37441540,75.0,78.0,78.0,1624.304133,2016-12-20,2018-06-20,0,281.264793,0.998764,...,0,0,0,0,0,1,0,0,0,0
2,84109342,66.0,77.0,77.0,1982.590000,2016-12-29,2018-06-19,0,250.556642,0.998598,...,0,0,0,0,0,0,0,1,0,0
3,127383407,77.0,78.0,78.0,1404.966753,2016-12-22,2018-06-20,0,288.677877,0.998795,...,0,0,0,0,0,0,0,0,1,0
4,178090012,71.0,77.0,77.0,1431.546620,2016-12-26,2018-06-18,0,269.287501,0.998695,...,0,0,0,0,0,0,0,0,0,0
